# <font size="7"> Datenbeschaffung für Wettervorhersagen <a class="anchor" id="02-D-03-wettervorhersagen"></a> </font>

<font size="5">
Die Daten werden von der Internetseite WeatherAPI.com bezogen und im Ordner „2-Data Understanding/Datenbeschaffung/Wettervorhersage/“ gespeichert. Die Daten wurden mittel einer Azure Logic App über die Forecast-API täglich um 23:59 heruntergeladen und fassen damit die Wettervorhersage für den jeweils nächsten Tag. Die Aufbereitung erfolgt wie bei den historischen Wetterdaten.
<br>Die Dokumentation der API ist unter folgendem Link erreichbar: https://www.weatherapi.com/docs/
</font>

In [ ]:
import pandas as pd

#Daten als Index vom 01.01.2015 bis 31.12.2021
indices = pd.date_range('07/31/2021', periods=153, freq='D')
#indices = pd.date_range('11/29/2018', periods=2, freq='D')

print("Von\t", indices[0])
print("Bis\t", indices[len(indices)-1])

#Städenamen und Koordinaten festlegen
cities = [
    ("Freiburg","48,7.85"),
    ("Mannheim","49.49,8.46"),
    ("Stuttgart","48.77,9.18"),
    ("Ulm","48.4010822,9.9876076")]

#city = "Freiburg"; coordinates = "48,7.85"
#city = "Mannheim"; coordinates = "49.49,8.46"
#city = "Stuttgart"; coordinates = "48.77,9.18"
#city = "Ulm"; coordinates = "48.4010822,9.9876076"

In [ ]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
import json

#Schleife, um Daten für alle Städte seriell zu verarbeiten
for city, coordinates in cities:
    
    print(city, "\t", coordinates)
    
    #DataFrame für Wetterdaten erstellen
    df_weather = pd.DataFrame()
    
    #Durchlaufen aller Tage
    for index in indices:
        
        #JSON-Datei für jeweiligen Tag laden
        file_path = "Wettervorhersage/" + city + "/" + index.date().strftime(format="%Y-%m-%d") + ".json"
        file = open(file_path, "r")
        
        #JSON-String auslesen
        try:
            content = json.loads(file.read())
        except:
            content = json.loads(open(file_path, 'r').read()[3:])
            
        #Wetterdaten für jeweiligen Tag in Zeile des DataFrames umwandeln
        df_day = pd.DataFrame.from_dict(content["forecast"]["forecastday"][0]["day"])
        df_day = df_day.iloc[2]
        
        #Astrodaten anhängen
        df_astro = pd.DataFrame.from_dict(content["forecast"]["forecastday"][0]["astro"], orient="index").transpose()
        df_day = df_day.append(df_astro.squeeze())

        #Zeile für jeweiligen Tag an DataFrame anhängen
        df_weather = df_weather.append(df_day, ignore_index=True)

        print(index.date())

    #Index setzen
    df_weather.set_index(pd.date_range('08/01/2021', periods=153, freq='D'), inplace=True)
    df_weather.index.name = "datum"
    
    #Wetterdaten bereinigen (Daten in Fahren/Meilen entfernen, Daten zum Mond entfernen)
    df_weather.drop(["avgtemp_f", "maxtemp_f", "mintemp_f", "avgvis_miles", "maxwind_mph", "totalprecip_in", "uv", "moonrise", "moonset", "moon_phase", "moon_illumination"], axis=1, inplace=True)
    df_weather.drop(["daily_chance_of_rain", "daily_chance_of_snow", "daily_will_it_rain", "daily_will_it_snow"], axis=1, inplace=True)
    #Spalten umbenennen
    df_weather.columns = ["luftfeuchtigkeit", "durchschnittstemperatur", "sichtweite", "wetter", "hoechsttemperatur", "windgeschwindigkeit", "tiefsttemperatur", "sonnenaufgang", "sonnenuntergang", "niederschlag"]
    
    #Wetterdaten für jeweilige Stadt speichern
    filename = (city + "_vorhersage.csv").lower()
    df_weather.to_csv(filename)